# Gradient Boosting with Pipeline

In [1]:
import pyspark
import sys

In [2]:
import pyspark.sql.functions as fn

In [3]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

In [4]:
# Check spark app name
spark.sparkContext.appName

'PySparkShell'

In [5]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

In [6]:
# print runtime versions
# Python version
sys.version

'3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]'

In [7]:
# Spark version
spark.version

'3.2.1'

### Exploring Data

In [ ]:
# load biking data into Spark dataframe
df = spark.read.csv('data/bike/hour.csv', header=True, inferSchema=True)

22/05/09 21:04:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/09 21:04:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/09 21:05:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/09 21:05:21 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/09 21:05:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/09 21:05:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

In [ ]:
#validate the size of data
df.count(), len(df.columns)

In [ ]:
# First 5 rows of Iris dataset
df.show(5)

In [ ]:
df.printSchema()

### Feature Engineering

In [ ]:
df.select('casual', 'registered', 'cnt').show(10)

In [ ]:
# discard 'instant', 'dteday', 'casual' and 'registered' columns
df = df.drop('instant').drop('dteday').drop('casual').drop('registered')

In [ ]:
df.show(10)

In [ ]:
df.printSchema()

### Split Data - Train & Test sets

In [ ]:
# use Logistic Regression to train on the training set
train_df, test_df = df.randomSplit([0.70, 0.30], seed=42)

In [ ]:
train_df.count(), len(train_df.columns)

In [ ]:
test_df.count(), len(test_df.columns)

### Build GBT Regression Model using Pipeline 

In [ ]:
featuresCols = df.columns[:-1]

In [ ]:
featuresCols

In [ ]:
# concatenates all feature columns into a single feature vector
vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="features")

In [ ]:
# Takes the "features" column and learns to predict "cnt"
gbt = GBTRegressor(labelCol="cnt")

In [ ]:
paramGrid = ParamGridBuilder()\
    .addGrid(gbt.maxDepth, [2, 5])\
    .addGrid(gbt.maxIter, [10, 100])\
    .build()

In [ ]:
# define evaluation metric  
# tell CrossValidator how well we are doing by comparing the true labels with predictions
evaluator = RegressionEvaluator(metricName="rmse", 
            labelCol=gbt.getLabelCol(), 
            predictionCol=gbt.getPredictionCol())

In [ ]:
# declare the CrossValidator, which runs model tuning for us
cv = CrossValidator(estimator=gbt, evaluator=evaluator, 
        estimatorParamMaps=paramGrid)

In [ ]:
# tie feature processing and model training stages together into a single Pipeline
pipeline = Pipeline(stages=[vectorAssembler, cv])

In [ ]:
%%time
# train the Pipeline
pipelineModel = pipeline.fit(train_df)

In [ ]:
pipelineModel.stages[1].bestModel._java_obj.getMaxDepth()

In [ ]:
pipelineModel.stages[1].bestModel._java_obj.getMaxIter()

### Evaluate Model

In [ ]:
# make predictions on test data 
predictions = pipelineModel.transform(test_df)

In [ ]:
# view the residual errors based on predictions 
predictions.select('cnt', 'prediction').show(10,False)

In [ ]:
# RMSE
rmse = evaluator.evaluate(predictions)

In [ ]:
rmse